In [1]:
from pandas import DataFrame, read_sql_query, Int32Dtype, read_excel
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
import pymysql
from sqlalchemy import create_engine, text
from decouple import config 
from dotenv import load_dotenv
InteractiveShell.ast_node_interactivity = "all"
from datim import datim_query

In [2]:
_QUERY="""
select * from patient;
"""
QUERY="""
SELECT
    p.patient_code,
    tmi.*
FROM
    tracking_motherbasicinfo tmi
LEFT JOIN
    patient p 
ON
    p.id = tmi.id_patient;
"""

MUSO_WITH_COMPTAGEMENAGE = """
SELECT *
FROM
    caris_db.view_muso_household_report
where (!isnull(indice_householdcount) or (is_household_applicable ='yes' ))
"""

""" INFO="mother_tracking_info"
_INFO="patient"
dataset.to_excel(f"../../pvvih_sansComptage/{_INFO}.xlsx",index=False) """

' INFO="mother_tracking_info"\n_INFO="patient"\ndataset.to_excel(f"../../pvvih_sansComptage/{_INFO}.xlsx",index=False) '

In [3]:
# Non-applicable 
MHC22 = f"""
SELECT 
    b.caseid, mhc22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.view_muso_household_report
    WHERE
        (!ISNULL(indice_householdcount)
            OR (is_household_applicable = 'yes'))) mhc22
        LEFT JOIN
    (SELECT 
        patient_code, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON mhc22.patient_code = b.patient_code
"""



# applicable 
NOMHC22 = f"""
SELECT 
    b.caseid, nomhc22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.view_muso_household_report
    WHERE
        (ISNULL(indice_householdcount)
            AND (is_household_applicable = 'no'
            OR ISNULL(is_household_applicable)))) nomhc22
        LEFT JOIN
    (SELECT 
        patient_code, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON nomhc22.patient_code = b.patient_code
"""

MUSO_HOUSEHOLD_22 = f"""
SELECT 
    b.caseid, mh22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.muso_household_2022) mh22
        LEFT JOIN
    (SELECT 
        id, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON mh22.id_patient = b.id
"""

# Non-applicable 
MHC22 = f"""
SELECT 
    b.caseid, mhc22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.view_muso_household_report
    WHERE
        (!ISNULL(indice_householdcount)
            OR (is_household_applicable = 'yes'))) mhc22
        LEFT JOIN
    (SELECT 
        patient_code, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON mhc22.patient_code = b.patient_code
"""



# applicable 
NOMHC22 = f"""
SELECT 
    b.caseid, nomhc22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.view_muso_household_report
    WHERE
        (ISNULL(indice_householdcount)
            AND (is_household_applicable = 'no'
            OR ISNULL(is_household_applicable)))) nomhc22
        LEFT JOIN
    (SELECT 
        patient_code, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON nomhc22.patient_code = b.patient_code
"""

MUSO_HOUSEHOLD_22 = f"""
SELECT 
    b.caseid, mh22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.muso_household_2022) mh22
        LEFT JOIN
    (SELECT 
        id, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON mh22.id_patient = b.id
"""

In [4]:
HUBS = f"""
SELECT 
    dh.name AS hub,
    dh.address,
    lc.name AS commune,
    ld.name AS departement,
    dh.is_inactive,
    dh.inactive_date,
    dh.creation_date,
    dh.created_at
FROM
    caris_db.dream_hub dh
        LEFT JOIN
    caris_db.lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    caris_db.lookup_departement ld ON ld.id = lc.departement
GROUP BY hub
HAVING hub != ''
"""

In [5]:
load_dotenv("../actual_caris_host.env")
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

True

In [6]:
mhc22 = read_sql_query(text(MHC22),engine.connect(),parse_dates=True)
nomhc22 = read_sql_query(text(NOMHC22),engine.connect(),parse_dates=True)
mh22 = read_sql_query(text(MUSO_HOUSEHOLD_22),engine.connect(),parse_dates=True)
#nomhc22 = read_sql_query(NOMHC22,engine,parse_dates=True)
#all_hub = read_sql_query(HUBS,engine,parse_dates=True)

In [7]:
engine.dispose()

In [8]:
mhc22.to_excel('./mhc22.xlsx',index=False)
nomhc22.to_excel('./nomhc22.xlsx',index=False)
mh22.to_excel('./mh22.xlsx',index=False)
#all_hub.to_excel('./all_hub.xlsx',index=False)